In [1220]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from IPython.display import display
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import MultiLabelBinarizer


In [1221]:
df =pd.read_csv("action.csv")

In [1222]:
print(df.index)

RangeIndex(start=0, stop=52452, step=1)


In [1223]:
df.head()

,movie_id,movie_name,year,certificate,runtime,genre,rating,description,director,director_id,star,star_id,votes,gross(in $)
0,tt9114286,Black Panther: Wakanda Forever,2022,PG-13,161 min,"Action, Adventure, Drama",6.9,The people of Wakanda fight to protect their h...,Ryan Coogler,/name/nm3363032/,"Letitia Wright, \nLupita Nyong'o, \nDanai Guri...","/name/nm4004793/,/name/nm2143282/,/name/nm1775...",204835.0,NaN
1,tt1630029,Avatar: The Way of Water,2022,PG-13,192 min,"Action, Adventure, Fantasy",7.8,Jake Sully lives with his newfound family form...,James Cameron,/name/nm0000116/,"Sam Worthington, \nZoe Saldana, \nSigourney We...","/name/nm0941777/,/name/nm0757855/,/name/nm0000...",295119.0,NaN
2,tt5884796,Plane,2023,R,107 min,"Action, Thriller",6.5,A pilot finds himself caught in a war zone aft...,Jean-François Richet,/name/nm0724938/,"Gerard Butler, \nMike Colter, \nTony Goldwyn, ...","/name/nm0124930/,/name/nm1591496/,/name/nm0001...",26220.0,NaN
3,tt6710474,Everything Everywhere All at Once,2022,R,139 min,"Action, Adventure, Comedy",8.0,A middle-aged Chinese immigrant is swept up in...,"Dan Kwan, \nDaniel Scheinert",/name/nm3453283/,"Michelle Yeoh, \nStephanie Hsu, \nJamie Lee Cu...","/name/nm3215397/,/name/nm0000706/,/name/nm3513...",327858.0,NaN
4,tt5433140,Fast X,2023,NaN,NaN,"Action, Crime, Mystery",NaN,Dom Toretto and his family are targeted by the...,Louis Leterrier,/name/nm0504642/,"Vin Diesel, \nJordana Brewster, \nTyrese Gibso...","/name/nm0004874/,/name/nm0108287/,/name/nm0879...",NaN,NaN


In [1224]:
df = df.drop(columns=["movie_name", "description","director","star"])

In [1225]:
df=df.dropna(subset=['rating'])

In [1226]:
df.head()

,movie_id,year,certificate,runtime,genre,rating,director_id,star_id,votes,gross(in $)
0,tt9114286,2022,PG-13,161 min,"Action, Adventure, Drama",6.9,/name/nm3363032/,"/name/nm4004793/,/name/nm2143282/,/name/nm1775...",204835.0,NaN
1,tt1630029,2022,PG-13,192 min,"Action, Adventure, Fantasy",7.8,/name/nm0000116/,"/name/nm0941777/,/name/nm0757855/,/name/nm0000...",295119.0,NaN
2,tt5884796,2023,R,107 min,"Action, Thriller",6.5,/name/nm0724938/,"/name/nm0124930/,/name/nm1591496/,/name/nm0001...",26220.0,NaN
3,tt6710474,2022,R,139 min,"Action, Adventure, Comedy",8.0,/name/nm3453283/,"/name/nm3215397/,/name/nm0000706/,/name/nm3513...",327858.0,NaN
5,tt10954600,2023,PG-13,125 min,"Action, Adventure, Comedy",6.6,/name/nm0715636/,"/name/nm0748620/,/name/nm1431940/,/name/nm3718...",5396.0,NaN


In [1227]:
df.dtypes

movie_id        object
year            object
certificate     object
runtime         object
genre           object
rating         float64
director_id     object
star_id         object
votes          float64
gross(in $)    float64
dtype: object

In [1228]:
unique_genres = set(genre.strip() for genres in df['genre'] for genre in genres.split(','))
print(unique_genres)

{'Musical', 'Fantasy', 'Horror', 'History', 'Sci-Fi', 'Action', 'War', 'Thriller', 'Crime', 'Biography', 'Drama', 'Romance', 'Family', 'Adventure', 'Sport', 'Animation', 'Music', 'Film-Noir', 'Mystery', 'Comedy', 'Western', 'Adult'}


In [1229]:
df['genre'] = df['genre'].apply(lambda x: x.split(', '))

# Używamy MultiLabelBinarizer
mlb = MultiLabelBinarizer()
binarized_data = mlb.fit_transform(df['genre'])

# Tworzymy nowy DataFrame z zerami i jedynkami
binarized_df = pd.DataFrame(binarized_data, columns=mlb.classes_)

# Wyświetlamy wynik
binarized_df.head()


,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Drama,Family,Fantasy,...,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
0,1,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,1,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [1230]:
binarized_df.fillna(0,inplace=True)

In [1231]:
df = df.drop(columns=["genre"])

In [1232]:
binarized_df.dtypes

Action       int32
Adult        int32
Adventure    int32
Animation    int32
Biography    int32
Comedy       int32
Crime        int32
Drama        int32
Family       int32
Fantasy      int32
Film-Noir    int32
History      int32
Horror       int32
Music        int32
Musical      int32
Mystery      int32
Romance      int32
Sci-Fi       int32
Sport        int32
Thriller     int32
War          int32
Western      int32
dtype: object

In [1233]:
print(len(df))

29448


In [1234]:
print(len(binarized_df))

29448


In [1235]:
print(df.index.equals(binarized_df.index))  # Sprawdza, czy indeksy są identyczne
print(df.index.difference(binarized_df.index))  # Wiersze, które są w df, ale nie w binarized_df
print(binarized_df.index.difference(df.index))  # Wiersze, które są w binarized_df, ale nie w df


False
Index([29448, 29449, 29451, 29452, 29454, 29458, 29459, 29460, 29461, 29462,
       ...
       52318, 52320, 52341, 52342, 52387, 52393, 52404, 52413, 52419, 52421],
      dtype='int64', length=5495)
Index([    4,    24,    29,    31,    36,    37,    41,    54,    59,    64,
       ...
       29423, 29424, 29429, 29437, 29439, 29441, 29443, 29444, 29446, 29447],
      dtype='int64', length=5495)


In [1236]:
binarized_df = binarized_df.set_index(df.index)


In [1237]:
 df=pd.concat([df, binarized_df], axis = 1)

In [1238]:
print(len(df))

29448


In [1239]:
filled4y= df['rating'].isna()
print(filled4y)

0        False
1        False
2        False
3        False
5        False
         ...  
52393    False
52404    False
52413    False
52419    False
52421    False
Name: rating, Length: 29448, dtype: bool


In [1240]:
binarized_df.head()

,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Drama,Family,Fantasy,...,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
0,1,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,1,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [1241]:
df.head()

,movie_id,year,certificate,runtime,rating,director_id,star_id,votes,gross(in $),Action,...,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
0,tt9114286,2022,PG-13,161 min,6.9,/name/nm3363032/,"/name/nm4004793/,/name/nm2143282/,/name/nm1775...",204835.0,NaN,1,...,0,0,0,0,0,0,0,0,0,0
1,tt1630029,2022,PG-13,192 min,7.8,/name/nm0000116/,"/name/nm0941777/,/name/nm0757855/,/name/nm0000...",295119.0,NaN,1,...,0,0,0,0,0,0,0,0,0,0
2,tt5884796,2023,R,107 min,6.5,/name/nm0724938/,"/name/nm0124930/,/name/nm1591496/,/name/nm0001...",26220.0,NaN,1,...,0,0,0,0,0,0,0,1,0,0
3,tt6710474,2022,R,139 min,8.0,/name/nm3453283/,"/name/nm3215397/,/name/nm0000706/,/name/nm3513...",327858.0,NaN,1,...,0,0,0,0,0,0,0,0,0,0
5,tt10954600,2023,PG-13,125 min,6.6,/name/nm0715636/,"/name/nm0748620/,/name/nm1431940/,/name/nm3718...",5396.0,NaN,1,...,0,0,0,0,0,0,0,0,0,0


In [1242]:
filled9= df['rating'].isna()
print(filled9)

0        False
1        False
2        False
3        False
5        False
         ...  
52393    False
52404    False
52413    False
52419    False
52421    False
Name: rating, Length: 29448, dtype: bool


In [1243]:
X = df.drop('rating', axis=1) 
y = pd.DataFrame(df, columns=['rating'])

In [1244]:
X.head()

,movie_id,year,certificate,runtime,director_id,star_id,votes,gross(in $),Action,Adult,...,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
0,tt9114286,2022,PG-13,161 min,/name/nm3363032/,"/name/nm4004793/,/name/nm2143282/,/name/nm1775...",204835.0,NaN,1,0,...,0,0,0,0,0,0,0,0,0,0
1,tt1630029,2022,PG-13,192 min,/name/nm0000116/,"/name/nm0941777/,/name/nm0757855/,/name/nm0000...",295119.0,NaN,1,0,...,0,0,0,0,0,0,0,0,0,0
2,tt5884796,2023,R,107 min,/name/nm0724938/,"/name/nm0124930/,/name/nm1591496/,/name/nm0001...",26220.0,NaN,1,0,...,0,0,0,0,0,0,0,1,0,0
3,tt6710474,2022,R,139 min,/name/nm3453283/,"/name/nm3215397/,/name/nm0000706/,/name/nm3513...",327858.0,NaN,1,0,...,0,0,0,0,0,0,0,0,0,0
5,tt10954600,2023,PG-13,125 min,/name/nm0715636/,"/name/nm0748620/,/name/nm1431940/,/name/nm3718...",5396.0,NaN,1,0,...,0,0,0,0,0,0,0,0,0,0


In [1245]:
X.dtypes

movie_id        object
year            object
certificate     object
runtime         object
director_id     object
star_id         object
votes          float64
gross(in $)    float64
Action           int32
Adult            int32
Adventure        int32
Animation        int32
Biography        int32
Comedy           int32
Crime            int32
Drama            int32
Family           int32
Fantasy          int32
Film-Noir        int32
History          int32
Horror           int32
Music            int32
Musical          int32
Mystery          int32
Romance          int32
Sci-Fi           int32
Sport            int32
Thriller         int32
War              int32
Western          int32
dtype: object

In [1246]:
y.head()

,rating
0,6.9
1,7.8
2,6.5
3,8.0
5,6.6


In [1247]:
le = LabelEncoder()
for col in X.columns:
    if X[col].dtype == 'object':
        X[col] = le.fit_transform(X[col].astype(str))

In [1248]:
X.dtypes

movie_id         int32
year             int32
certificate      int32
runtime          int32
director_id      int32
star_id          int32
votes          float64
gross(in $)    float64
Action           int32
Adult            int32
Adventure        int32
Animation        int32
Biography        int32
Comedy           int32
Crime            int32
Drama            int32
Family           int32
Fantasy          int32
Film-Noir        int32
History          int32
Horror           int32
Music            int32
Musical          int32
Mystery          int32
Romance          int32
Sci-Fi           int32
Sport            int32
Thriller         int32
War              int32
Western          int32
dtype: object

In [1249]:
nan = X['gross(in $)'].isna().sum()
print(nan)

26688


In [1250]:
filled= X['gross(in $)'].count()
print(filled)

2760


In [1251]:
X.fillna({'gross(in $)':X['gross(in $)'].median()}, inplace=True)
X['gross(in $)'].head()

0    10903645.5
1    10903645.5
2    10903645.5
3    10903645.5
5    10903645.5
Name: gross(in $), dtype: float64

In [1252]:
czy_nan_w_df = X.isna().any().any()
print(czy_nan_w_df)  # True, jeśli gdziekolwiek w DF jest NaN


False


In [1253]:
czy_nan_w_df2 = y.isna().any().any()
print(czy_nan_w_df2)  # True, jeśli gdziekolwiek w DF jest NaN

False


In [1254]:
filled3= y['rating'].isna().sum()
print(filled3)

0


In [1255]:
y.fillna({'rating':y['rating'].median()}, inplace=True)
y['rating'].head()

0    6.9
1    7.8
2    6.5
3    8.0
5    6.6
Name: rating, dtype: float64

In [1256]:
kolumny_z_nan = X.isna().any()
print(kolumny_z_nan)  # Lista kolumn z NaN


movie_id       False
year           False
certificate    False
runtime        False
director_id    False
star_id        False
votes          False
gross(in $)    False
Action         False
Adult          False
Adventure      False
Animation      False
Biography      False
Comedy         False
Crime          False
Drama          False
Family         False
Fantasy        False
Film-Noir      False
History        False
Horror         False
Music          False
Musical        False
Mystery        False
Romance        False
Sci-Fi         False
Sport          False
Thriller       False
War            False
Western        False
dtype: bool


In [1257]:
print(type(kolumny_z_nan))

<class 'pandas.core.series.Series'>


In [1258]:
X.dtypes

movie_id         int32
year             int32
certificate      int32
runtime          int32
director_id      int32
star_id          int32
votes          float64
gross(in $)    float64
Action           int32
Adult            int32
Adventure        int32
Animation        int32
Biography        int32
Comedy           int32
Crime            int32
Drama            int32
Family           int32
Fantasy          int32
Film-Noir        int32
History          int32
Horror           int32
Music            int32
Musical          int32
Mystery          int32
Romance          int32
Sci-Fi           int32
Sport            int32
Thriller         int32
War              int32
Western          int32
dtype: object

In [1259]:
y.head()

,rating
0,6.9
1,7.8
2,6.5
3,8.0
5,6.6


In [1260]:
X['director_id'] = X['director_id'].astype('category').cat.codes

In [1261]:
le = LabelEncoder()
X["director_id"] = le.fit_transform(X["director_id"])


In [1262]:
X.head()

,movie_id,year,certificate,runtime,director_id,star_id,votes,gross(in $),Action,Adult,...,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
0,29117,113,15,64,11718,25719,204835.0,10903645.5,1,0,...,0,0,0,0,0,0,0,0,0,0
1,20249,113,15,95,8,17912,295119.0,10903645.5,1,0,...,0,0,0,0,0,0,0,0,0,0
2,26951,114,17,9,5320,6144,26220.0,10903645.5,1,0,...,0,0,0,0,0,0,0,1,0,0
3,27639,113,17,41,11835,24593,327858.0,10903645.5,1,0,...,0,0,0,0,0,0,0,0,0,0
5,16835,114,15,27,5263,15336,5396.0,10903645.5,1,0,...,0,0,0,0,0,0,0,0,0,0
